In [46]:
import numpy as np
from tqdm.auto import tqdm
from time import time
import json, pickle, os, string, kenlm, json
from collections import defaultdict, Counter
from itertools import groupby
import Levenshtein as Lev
import math


In [47]:
def lse(*args):
  """
  Stable log sum exp.
  """
  if all(a == -float('inf') for a in args):
      return -float('inf')
  a_max = max(args)
  lsp = math.log(sum(math.exp(a - a_max)
                      for a in args))
  return a_max + lsp

In [48]:
#s1 = True text
#s2 = predicted text

def wer_(s1, s2):
    """
    Computes the Word Error Rate, defined as the edit distance between the
    two provided sentences after tokenizing to words.
    Arguments:
        s1 (string): space-separated sentence
        s2 (string): space-separated sentence
    """

    # build mapping of words to integers
    b = set(s1.split() + s2.split())
    word2char = dict(zip(b, range(len(b))))

    # map the words to a char array (Levenshtein packages only accepts
    # strings)
    w1 = [chr(word2char[w]) for w in s1.split()]
    w2 = [chr(word2char[w]) for w in s2.split()]
    
    return Lev.distance(''.join(w1), ''.join(w2))

def cer_(s1, s2):
    """
    Computes the Character Error Rate, defined as the edit distance.

    Arguments:
        s1 (string): space-separated sentence
        s2 (string): space-separated sentence
    """
    s1, s2, = s1.replace(' ', ''), s2.replace(' ', '')

    return Lev.distance(s1, s2)




#When using the above implementation, use the code belove to calculate the wer in percentatge: 
#pred = list of ouput prediction of model (it is the text) # example [" MY NAME IS HEMANT", " I AM A GOD"]
# total_wer = 0
# for x in range(len(pred)):
#     transcript, reference = data_[x][1], pred[x]
#     wer_inst = wer(transcript, reference)
#     total_wer += float(wer_inst)
# print("WER is : ",total_wer/len(pred),"%")
# wer_(pred,true)/len(pred.split(' '))*100

In [49]:
out = np.load("out.npy")[0]
with open("true.txt", "r") as f:
    reference = f.read().strip()
    
with open("pred.txt", "r") as f:
    transcript = f.read().strip()

In [50]:
def ctc_best_path(out,labels):
    "implements best path decoding as shown by Graves"
    out = [labels[i] for i in np.argmax(out, axis=1) if i!=labels[-1]]
    o = ""
    for i,j in groupby(out):
        o = o + i
    return o.replace("_","")

In [51]:
labels = "_'ABCDEFGHIJKLMNOPQRSTUVWXYZ "

In [52]:
gred_txt = ctc_best_path(out,labels)
print(gred_txt)
np.round(wer_(gred_txt,reference)/len(reference.split(' '))*100,3)

AND CHARGED IFEVER HE MIGHT FIND SIR GAWANE AND SIR UWANE TO BRING THEM TO THE COURT AGAIN AND THEN WERE THEY ALL GLAD AND SO PRAY DHAS OR MORE HOUSE TO RIDE WITH THEM TO THE KING'S COURT


23.077

### WORD LM Implementation

In [198]:
lm_w = kenlm.LanguageModel('/home/hemant/asr_wm/data/ner/align_wav_txt/4_gram.arpa')

In [199]:
def sort_beam(ptot,k):
    if len(ptot) < k:
        return [i for i in ptot.keys()]
    else:
        dict_ = sorted(dict((v,k) for k,v in ptot.items()).items(),reverse=True)[:k]
        return [i[1] for i in dict_]


In [234]:
def ctc_beam_search(out,labels, prune=0.0001, k=20, lm=None,alpha=0.3,beta=12):
    
    bc_i = 0 # blank/special charatcter index 
    F = out.shape[1]
    dummy_ = np.vstack((np.zeros(F), out))
    out = np.log(np.vstack((np.zeros(F), out)))
    steps = out.shape[0]
    
    pb, pnb = defaultdict(Counter), defaultdict(Counter)
    pb[0][''], pnb[0][''] = 0, -float("inf")
    prev_beams = ['']
    for t in tqdm(range(1,steps)):
        pruned_alphabet = [labels[i] for i in np.where(dummy_[t] > prune)[0]]
        for b in prev_beams:
            
            for c_t in pruned_alphabet:
                index = labels.index(c_t)
                if c_t == "_": #Extending with a blank
                    pb[t][b] = lse(pb[t][b],out[t][index]+pb[t-1][b], out[t][index]+pnb[t-1][b])
                    
                    continue
                
                else:
                    i_plus = b + c_t
                    if len(b.replace(' ', '')) > 0  and c_t == b[-1]: #Extending with the same character as the last one
                        pnb[t][b] = lse(pnb[t][b],out[t][index]+pnb[t-1][b])
                        pnb[t][i_plus] = lse(pnb[t][i_plus],out[t][index]+pb[t-1][b])
                    elif c_t == " " and len(b.replace(' ', '')) > 0 : # LM constraints
                        words = b.strip()
                        prob = 10**[i for i in lm.full_scores(words,eos=False,bos=False)][-1][0]
                        prob = alpha*lse(math.log(prob))
                        word_inser = len(words.split())+1
                        word_inser = beta*lse(math.log(word_inser))
                        score_ = prob + word_inser
                        pnb[t][i_plus] = lse(out[t][index]+pb[t-1][b],out[t][index] + pnb[t-1][b]) + score_
                        
                        print(words,pnb[t][i_plus],',',prob,word_inser,score_)
#                         pnb[t][i_plus] = pnb[t][i_plus] + score_
                    else:
                        pnb[t][i_plus] = lse(pnb[t][i_plus],out[t][index]+pb[t-1][b], out[t][index]+pnb[t-1][b])

#                     If the new beam is not in the previous beams
                    if i_plus not in prev_beams:
                        pb[t][i_plus] = lse(pb[t][i_plus],out[t][labels.index("_")]+pb[t - 1][i_plus], out[t][labels.index("_")]+ pnb[t - 1][i_plus])
                        pnb[t][i_plus] = lse(pnb[t][i_plus],out[t][index] + pnb[t - 1][i_plus])

        ptot = pb[t] + pnb[t]
        prev_beams = sort_beam(ptot,k)
#         print('\n')
        
    return prev_beams[0], pb, pnb


In [ ]:
beam_txt, pb, pnb = ctc_beam_search(out[:],labels,0.000,k=100,lm=lm_w,alpha=0.58,beta=3)

/home/hemant/.conda/envs/deep/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


In [246]:
print(beam_txt)
np.round(wer_(beam_txt,reference)/len(reference.split(' '))*100,3)

STAND  CHARGED  IFEVER HE  MIGHT FIND SIR GAWAINE AND SIR U WAIYINE TO BRING THEM TO THE COAURT A GAIN  AND THEN I WERE THEY ALL GLAD D  AND SO I A PRAIYE DE I S S OR MO RE HO USE S TO RI DE WITH HE M TO THE KING'S COOURT E 


74.359

In [247]:
len(beam_txt.split()), len(reference.split())

(54, 39)

In [248]:
reference

"AND CHARGED IF EVER HE MIGHT FIND SIR GAWAINE AND SIR UWAINE TO BRING THEM TO THE COURT AGAIN AND THEN WERE THEY ALL GLAD AND SO PRAYED THEY SIR MARHAUS TO RIDE WITH THEM TO THE KING'S COURT"

In [ ]:
import os
os.chdir("/home/hemant/E2E_NER-Through-Speech/S2T/")
from opts import add_decoder_args, add_inference_args
from utils import load_model
import os
# from ctc_decoders import *
import argparse

import numpy as np
import torch
from tqdm import tqdm
from data.data_loader import SpectrogramParser

from opts import add_decoder_args, add_inference_args
from utils import load_model

In [ ]:
decoding = 'beam_w'
prune = 0.00001
beam_width = 50
alpha = 1
beta = 4
lm = kenlm.LanguageModel('/home/hemant/asr_wm/data/ner/align_wav_txt/4_gram.arpa')

torch.set_grad_enabled(False)
device = torch.device("cuda")
model = load_model(device, "/home/hemant/asr_wm/models/deep/final.pth")
spect_parser = SpectrogramParser(model.audio_conf, normalize=True)

torch.cuda.set_device(int(0))
with open("/home/hemant/asr_wm/dev.csv","r") as f:
    csv = f.readlines()

total_cer, total_wer, num_tokens, num_chars = 0, 0, 0, 0
lm = lm_w
output = []
from tqdm.auto import tqdm
for i in tqdm(csv[:1]):
    audio_path, reference_path = i.split(",")

    spect = spect_parser.parse_audio(audio_path).contiguous()
    spect = spect.view(1, 1, spect.size(0), spect.size(1))
    spect = spect.to(device)

    input_sizes = torch.IntTensor([spect.size(3)]).int()
    out, output_sizes = model(spect, input_sizes)
    out = out.cpu().detach().numpy()[0]

    transcript = ctc_beam_search(out,labels,prune,beam_width,lm,alpha,beta)
#     transcript = ctc_best_path(out,labels)
    with open(reference_path.replace("\n",""),"r") as f:
        reference = f.readline()
        
#     break
    output.append([transcript,reference])
    wer_inst = wer_(transcript,reference)
    cer_inst = cer_(transcript, reference)
    total_wer += wer_inst
    total_cer += cer_inst
    num_tokens += len(reference.split(' '))
    num_chars += len(reference.replace(' ', ''))

        
wer = (float(total_wer) / num_tokens)*100
cer = (float(total_cer) / num_chars)*100
print('Test Summary \t'
    'Average WER {wer:.3f}\t'
    'Average CER {cer:.3f}\t?'.format(wer=wer, cer=cer))

In [ ]:
alpha, beta = np.linspace(0.2,2,15), np.linspace(2,10,5)

In [ ]:
alpha

In [ ]:
beta

In [ ]:
alpha, beta = np.linspace(0.2,2,15), np.linspace(0,6,15)
values = [[i,j] for j in beta for i in alpha]

total_cer, total_wer, num_tokens, num_chars = 0, 0, 0, 0
lm = lm_w
output = []

for alpha,beta in tqdm(values):
    for i in csv:
        audio_path, reference_path = i.split(",")

        spect = spect_parser.parse_audio(audio_path).contiguous()
        spect = spect.view(1, 1, spect.size(0), spect.size(1))
        spect = spect.to(device)

        input_sizes = torch.IntTensor([spect.size(3)]).int()
        out, output_sizes = model(spect, input_sizes)
        out = out.cpu().detach().numpy()[0]

        if decoding == "greedy": transcript = ctc_best_path(out,labels)
        elif decoding == "beam_w": transcript = ctc_beam_search(out,labels,prune,beam_width,lm,alpha,beta)
        elif decoding == "beam_c": transcript = ctc_beam_search_clm(out,labels,prune,beam_width,lm,alpha=alpha,beta=beta)

        with open(reference_path.replace("\n",""),"r") as f:
            reference = f.readline()
        wer_inst = wer_(transcript,reference)
        cer_inst = cer_(transcript, reference)
        total_wer += wer_inst
        total_cer += cer_inst
        num_tokens += len(reference.split(' '))
        num_chars += len(reference.replace(' ', ''))
    np.save("/home/hemant/ctc_decoders/abw.npy",np.array(output))
    wer = (float(total_wer) / num_tokens)*100
    cer = (float(total_cer) / num_chars)*100
    output.append([alpha,beta,wer])
    print("aplha: ",alpha,"beta: ",beta)
    print('Test Summary \t'
        'Average WER {wer:.3f}\t'
        'Average CER {cer:.3f}\t'.format(wer=wer, cer=cer))